In [ ]:
def clouds_shadows_mask_L7(image):
    qa = image.select('pixel_qa')
#   If the cloud bit (5) is set and the cloud confidence (7) is high
#   or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = (qa.bitwiseAnd(1 << 5)
             .And(qa.bitwiseAnd(1 << 7))
             .Or(qa.bitwiseAnd(1 << 3)))
#  emove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    mask = image.updateMask(cloud.Not()).updateMask(mask2)
    return mask


def clouds_shadows_mask_L8(image):
#   // Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
#   // Get the pixel QA band.
    qa = image.select('pixel_qa');
#   // Both flags should be set to zero, indicating clear conditions.
    mask2 = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
             .And(qa.bitwiseAnd(cloudsBitMask).eq(0)))
    mask = image.updateMask(mask2) 
    return mask

#function reflectance 
def reflectance(image):
    return ee.Image(image.multiply(0.0001).copyProperties(image,["system:time_start"]))

bandNames = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

def VIs_L7(image):
  IVs = (image.addBands(image.normalizedDifference(['NIR', 'RED']).rename('NDVI')))
#     IVs = IVs.addBands(image.normalizedDifference(['B4', 'B5']).rename('NDRE'))
  IVs = (IVs.addBands(image.normalizedDifference(['NIR', 'B2']).rename('GNDVI')))
  IVs = (IVs.addBands(image.normalizedDifference(['B2', 'B5']).rename('DNWVI')))
  IVs = (IVs.addBands(image.normalizedDifference(['B5', 'NIR']).rename('NWVI')))
  IVs = (IVs.addBands(image.expression(
  '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
    'NIR': image.select('B4'),
    'RED': image.select('RED'),
    'BLUE': image.select('B1')}).rename('EVI')))
#     IVs = (IVs.addBands(image.expression(
#     '(NIR * (1 - RED) * (NIR - RED))^(1/3)', {
#       'NIR': image.select('B4'),
#       'RED': image.select('B3')}).rename('AVI')))
  IVs = (IVs.addBands(image.expression(
  '(NIR - 2 * RED + BLUE)/(NIR + 2 * RED - BLUE)', {
    'NIR': image.select('B4'),
    'RED': image.select('RED'),
    'BLUE': image.select('B1')}).rename('ARVI')))
  IVs =( IVs.addBands(image.expression(
  'NIR/(NIR + RED)', {
    'NIR': image.select('B4'),
    'RED': image.select('RED'),}).rename('IPVI')))
  IVs = (IVs.addBands(image.expression(
  '(NIR-RED)/(NIR+RED + 0.16)', {
    'NIR': image.select('B4'),
    'RED': image.select('B3'),}).rename('OSAVI')))
  IVs = (IVs.addBands(image.expression(
  'GREEN - 0.39 * RED - 0.69 * BLUE', {
    'BLUE': image.select('B1'),
    'RED': image.select('B3'),
    'GREEN': image.select('B2'),}).rename('TGI')))
  return IVs

def VIs_L8(image):
  IVs = (image.addBands(image.normalizedDifference(['B5', 'B4']).rename('NDVI')))
#     IVs = IVs.addBands(image.normalizedDifference(['B4', 'B5']).rename('NDRE'))
  IVs = (IVs.addBands(image.normalizedDifference(['B5', 'B2']).rename('GNDVI')))
  IVs = (IVs.addBands(image.normalizedDifference(['B2', 'B6']).rename('DNWVI')))
  IVs = (IVs.addBands(image.normalizedDifference(['B6', 'B5']).rename('NWVI')))
  IVs = (IVs.addBands(image.expression(
  '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
    'NIR': image.select('B5'),
    'RED': image.select('B4'),
    'BLUE': image.select('B2')}).rename('EVI')))
#     IVs = (IVs.addBands(image.expression(
#     '(NIR * (1 - RED) * (NIR - RED))^(1/3)', {
#       'NIR': image.select('B5'),
#       'RED': image.select('B4')}).rename('AVI')))
  IVs = (IVs.addBands(image.expression(
  '(NIR - 2 * RED + BLUE)/(NIR + 2 * RED - BLUE)', {
    'NIR': image.select('B5'),
    'RED': image.select('B4'),
    'BLUE': image.select('B2')}).rename('ARVI')))
  IVs =( IVs.addBands(image.expression(
  'NIR/(NIR + RED)', {
    'NIR': image.select('B5'),
    'RED': image.select('B4'),}).rename('IPVI')))
  IVs = (IVs.addBands(image.expression(
  '(NIR-RED)/(NIR+RED + 0.16)', {
    'NIR': image.select('B5'),
    'RED': image.select('B4'),}).rename('OSAVI')))
  IVs = (IVs.addBands(image.expression(
  'GREEN - 0.39 * RED - 0.69 * BLUE', {
    'BLUE': image.select('B2'),
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),}).rename('TGI')))
  return IVs